Load the Document


In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../Corpus/The Godfather Summary.pdf")
documents = loader.load()

Split Documents


In [2]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50,separator="\n")
texts = text_splitter.split_documents(documents)

Create Embeddings


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/var/folders/pz/45l1b5fj5jn_1q55dmlkmj_h0000gn/T/ipykernel_57035/312703518.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/Users/dhruv/Documents/Lambton College /2 Semester/Natural Language Processing/Final_Project/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector Store (FAISS)


In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env.token")
huggingface_token = os.getenv("YOUR_HUGGINGFACE_API_TOKEN")

if not huggingface_token:
    raise ValueError("Hugging Face API token not found in .env.token file.")


LLM


In [ ]:
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.25, "max_length": 512},
    huggingfacehub_api_token=huggingface_token
    )

Prompt Template


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT
**********************************************************************
{context}
**********************************************************************


Question: {question}

Answer:"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


RetrievalQA Chain


In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},  # Pass the custom prompt
)

In [ ]:
query = "What are the main conflicts or storylines in the document?"
result = qa.run(query)
print(result)

/Users/dhruv/Documents/Lambton College /2 Semester/Natural Language Processing/Final_Project/.env/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

**********************************************************************
Context
House of Atreus. Coppola cuts from the younger Vito to 
his successor, Michael — and whatever glow Michael got 
when he reached power fades as he sets about consoli-
dating it. The Don’s legacy of hypocrisy and crime eats 
away at Michael’s soul. 
 
These two movies together are not really about the dete-
rioration of the American dream. What they say is that 
for immigrant groups that became the country’s back-
bone — Italians, Jews, Irish, and others — the American 
Dream was limited from the start by the burdens of pov-
erty, unsettled scores, and insular ethnic cultures. As in 
the Old World, they were prey to powerful economic and 
political forces. But here those forces took more various, 
insidious forms. Many Vietnam-era movies told us tha